In [1]:
import numpy as np
import torch
from torch.nn import Sequential, ReLU, Linear, Conv1d, LSTM, RNN, Dropout
from torch.nn import Module
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor   
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

from NCP.model import DeepSVD
from NCP.layers import SingularLayer, MLPOperator

np.random.seed(0)
torch.manual_seed(0)
Ntrain = 1000
Nval = 1000
Ntest = 1000

ImportError: cannot import name 'cme_score' from 'NCP.nn.functional' (/Users/gregoirepacreau/Documents/NCP/NCP/nn/functional.py)

In [ ]:

X = np.random.uniform(-5, 5, Ntrain+Nval+Ntest)
Y = 5*X+3 + np.random.normal(0,1, X.shape[0])


X = X.reshape((-1, 1))
Y = Y.reshape((-1, 1))

xscaler = StandardScaler()
yscaler = StandardScaler()
Xtransformed = xscaler.fit_transform(X)
Ytransformed = yscaler.fit_transform(Y)

X_train, X_test, Y_train, Y_test = train_test_split(Xtransformed, Ytransformed, test_size=Ntest, random_state=1)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=Nval, random_state=1)

In [ ]:
plt.scatter(X_train, Y_train, alpha=0.5)
plt.scatter(X_val, Y_val, color='green', alpha=0.3)
plt.scatter(X_test, Y_test, color='red', alpha=0.3)
plt.legend(['train', 'val', 'test'])
plt.show()

In [ ]:
input_shape = X_train.shape[-1]
output_shape = 2
n_hidden = 0
layer_size = []

U_operator = MLPOperator(input_shape, n_hidden, layer_size, output_shape)
V_operator = MLPOperator(input_shape, n_hidden, layer_size, output_shape)

optimizer = Adam
optimizer_kwargs = {
    'lr': 1e-3
    }

In [ ]:
reg = DeepSVD(U_operator, V_operator, output_shape, gamma=0.)
reg.fit(X_train, Y_train, X_val, Y_val, epochs=int(1e5), optimizer=optimizer, optimizer_kwargs=optimizer_kwargs)

ls = reg.get_losses()
plt.figure(figsize=(17, 8))
plt.scatter(range(len(ls)), np.array(ls))
plt.scatter(range(len(ls)), np.array(reg.get_val_losses()), alpha=0.3)
plt.yscale('log')

In [ ]:
pred = reg.predict(X_train).reshape(-1, 1)

plt.scatter(X_train, Y_train, color='r', alpha=0.3)
plt.scatter(X_train, pred, alpha=0.5)
plt.show()